# title

In [87]:
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [89]:
def get_video_urls(youtube_playlist_url):
    #webdriver setup & html scrape
    driver = webdriver.Chrome('resources/chromedriver')
    driver.get(youtube_playlist_url)
    soup = BeautifulSoup(driver.page_source)

    #isolate the video stats
    video_stats = soup.find('div', id='stats').text
    #use regex to isolate video number
    no_of_videos = re.search('(\d{1,3},?\d{1,3},?)', video_stats).group()
    #number of scroll based on 100 thumbnails per scroll
    no_of_scrolls = int(int(no_of_videos) / 100) + 1

    #scroll page for every 100 videos
    for i in range(no_of_scrolls):
        driver.execute_script("window.scrollBy(0, 12000);")
        time.sleep(3)

    soup = BeautifulSoup(driver.page_source)
    driver.quit()

    #extract youtube playlist urls
    matches = soup.find_all('a', class_='yt-simple-endpoint style-scope ytd-playlist-video-renderer')
    youtube_urls = []
    for url in matches:
        string = url.get('href')
        substring = re.search(r'\/watch\?v=\w*', string).group()
        youtube_urls.append('http://www.youtube.com' + substring)    
    return youtube_urls

In [90]:
gerard = get_video_urls('https://www.youtube.com/playlist?list=PLeCdlPO-XhWFzEVynMsmosfdRsIZXhZi0')

In [94]:
for index, url in enumerate(gerard):
    print(index + 1, url)

1 http://www.youtube.com/watch?v=RgKAFK5djSk
2 http://www.youtube.com/watch?v=OPf0YbXqDm0
3 http://www.youtube.com/watch?v=fRh_vgS2dFE
4 http://www.youtube.com/watch?v=09R8_2nJtjg
5 http://www.youtube.com/watch?v=CevxZvSJLk8
6 http://www.youtube.com/watch?v=lp
7 http://www.youtube.com/watch?v=hT_nvWreIhg
8 http://www.youtube.com/watch?v=nfWlot6h_JM
9 http://www.youtube.com/watch?v=NUsoVlDFqZg
10 http://www.youtube.com/watch?v=YqeW9_5kURI
11 http://www.youtube.com/watch?v=0KSOMA3QBU0
12 http://www.youtube.com/watch?v=60ItHLz5WEA
13 http://www.youtube.com/watch?v=e
14 http://www.youtube.com/watch?v=PT2_F
15 http://www.youtube.com/watch?v=kOkQ4T5WO9E
16 http://www.youtube.com/watch?v=papuvlVeZg8
17 http://www.youtube.com/watch?v=AJtDXIazrMo
18 http://www.youtube.com/watch?v=DK_0jXPuIr0
19 http://www.youtube.com/watch?v=pXRviuL6vMY
20 http://www.youtube.com/watch?v=uelHwf8o7_U
21 http://www.youtube.com/watch?v=IcrbM1l_BoI
22 http://www.youtube.com/watch?v=YBHQbu5rbdQ
23 http://www.youtube.

- title
- url
- account name
- subscribers
- views
- date uploaded
- number of comments
- number of likes
- number of dislikes